In [1]:
!pip install -q torch torchvision torchaudio trl peft accelerate bitsandbytes unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [27]:
import gc
model = ""
tokenizer = ""
gc.collect()
torch.cuda.empty_cache()

In [28]:
import torch
from unsloth import FastLanguageModel

# Загружаем модель Qwen2-4B.
# ВАЖНО: load_in_4bit = False, dtype = torch.float16
# Это требует значительно больше видеопамяти.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "RefalMachine/RuadaptQwen3-4B-Instruct",
    max_seq_length = 2048,
    dtype = torch.float16,   # Указываем тип данных float16
    load_in_4bit = False,
    device_map="balanced"
)

# Настраиваем модель для PEFT (Parameter-Efficient Fine-Tuning) с использованием LoRA.
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
)

==((====))==  Unsloth 2025.11.2: Fast Qwen3 patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [29]:
import ast
from datasets import load_dataset

dataset = load_dataset("csv", data_files="/kaggle/input/sirius-formated/llm_ready_recipe_dataset.csv", split="train")
print(f"Датасет успешно загружен. Количество записей: {len(dataset)}")

# Определяем неизменяемую часть промптов
SYSTEM_PROMPT = """Ты — ИИ-ассистент для системы продуктовых рекомендаций.
Твоя задача — анализировать корзину и генерировать идеи для поиска недостающих товаров в виде списка поисковых запросов."""

USER_PROMPT_TEMPLATE = """Контекст:
    Текущая корзина: {current_basket}
    Прошлые 5 покупок пользователя:
    Прошлые 5 наиболее похожих покупок пользователя:
### ЗАДАЧА
На основе КОНТЕКСТА, сгенерируй не более 20 поисковых запросов, которые помогут пользователю добавить недостающие товары в корзину.
Важные правила:
1. Запросы должны быть краткими, отражать общие категории или идеи, а не конкретными товарами с брендом или весом.
2. Не повторяй товары, которые уже есть в корзине.
3. Поисковые запросы должны быть реалистичными для продуктового магазина с 10-15 тысячами наименований
4. Вывод должен быть в формате [<запрос 1>, <запрос 2>, ...]

### ПРИМЕР (для демонстрации логики, а не для копирования)
- Пример входной корзины: [Мука пшеничная; Яйца куриные; Сахар-песок]
- Пример правильного вывода: ["разрыхлитель", "ванильный экстракт", "сливочное масло", "шоколад", "кондитерские украшения"]"""

def format_data_as_messages(example):
    try:
        input_basket_list = ast.literal_eval(example["input_basket"])
        target_basket_list = ast.literal_eval(example["target_basket"])
    except (ValueError, SyntaxError):
        return {"text": None}

    # 1. Собираем контент для каждого сообщения
    current_basket_str = "; ".join(input_basket_list)
    user_content = USER_PROMPT_TEMPLATE.format(current_basket=current_basket_str)
    assistant_content = str(target_basket_list)

    # 2. Создаем структуру messages
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_content},
        {"role": "assistant", "content": assistant_content},
    ]
    # 3. Используем токенизатор для преобразования messages в одну строку для обучения
    # add_generation_prompt=False, так как мы предоставляем и ответ ассистента
    return { "text": tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False) }

# Применяем новую функцию форматирования
dataset = dataset.map(format_data_as_messages)
dataset = dataset.filter(lambda example: example["text"] is not None)
print("Данные отформатированы с использованием chat-шаблона.")

Датасет успешно загружен. Количество записей: 123308
Данные отформатированы с использованием chat-шаблона.


In [30]:
dataset[0]

{'recipe_url': 'https://www.povarenok.ru/recipes/show/1306/',
 'input_basket': "['Яйцо куриное', 'Грейпфрут', 'Салат', 'Майонез']",
 'target_basket': "['Сыр твердый', 'Чеснок', 'Лук зеленый']",
 'text': '<|im_start|>system\nТы — ИИ-ассистент для системы продуктовых рекомендаций.\nТвоя задача — анализировать корзину и генерировать идеи для поиска недостающих товаров в виде списка поисковых запросов.<|im_end|>\n<|im_start|>user\nКонтекст:\n    Текущая корзина: Яйцо куриное; Грейпфрут; Салат; Майонез\n    Прошлые 5 покупок пользователя:\n    Прошлые 5 наиболее похожих покупок пользователя:\n### ЗАДАЧА\nНа основе КОНТЕКСТА, сгенерируй не более 20 поисковых запросов, которые помогут пользователю добавить недостающие товары в корзину.\nВажные правила:\n1. Запросы должны быть краткими, отражать общие категории или идеи, а не конкретными товарами с брендом или весом.\n2. Не повторяй товары, которые уже есть в корзине.\n3. Поисковые запросы должны быть реалистичными для продуктового магазина с 

In [31]:
# с данными что-то не то, надо чинить. Ну и вообще как будто стоит взять ноут свой рабочий да адаптировать, а не хуйней заниматься

In [32]:
print("Первый пример из датасета:")
print(dataset[0]["text"])
print(f"\nДлина текста: {len(dataset[0]['text'])}")

# Проверка токенизации
sample = dataset[0]["text"]
tokens = tokenizer(sample, truncation=True, max_length=2048)
print(f"Количество токенов: {len(tokens['input_ids'])}")

Первый пример из датасета:
<|im_start|>system
Ты — ИИ-ассистент для системы продуктовых рекомендаций.
Твоя задача — анализировать корзину и генерировать идеи для поиска недостающих товаров в виде списка поисковых запросов.<|im_end|>
<|im_start|>user
Контекст:
    Текущая корзина: Яйцо куриное; Грейпфрут; Салат; Майонез
    Прошлые 5 покупок пользователя:
    Прошлые 5 наиболее похожих покупок пользователя:
### ЗАДАЧА
На основе КОНТЕКСТА, сгенерируй не более 20 поисковых запросов, которые помогут пользователю добавить недостающие товары в корзину.
Важные правила:
1. Запросы должны быть краткими, отражать общие категории или идеи, а не конкретными товарами с брендом или весом.
2. Не повторяй товары, которые уже есть в корзине.
3. Поисковые запросы должны быть реалистичными для продуктового магазина с 10-15 тысячами наименований
4. Вывод должен быть в формате [<запрос 1>, <запрос 2>, ...]

### ПРИМЕР (для демонстрации логики, а не для копирования)
- Пример входной корзины: [Мука пшеничная

In [ ]:
from trl import SFTTrainer, SFTConfig
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,   # dataset с полем "text" (строки)
    eval_dataset = None,
    args = SFTConfig(
        dataset_text_field = "text",           # <- важное отличие
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 16,
        warmup_steps = 5,
        num_train_epochs=1,                    # или max_steps/num_train_epochs по вашей задаче
        learning_rate = 2e-4,
        logging_steps = 10,
        optim = "adamw_8bit",                  # советуют в ноуте
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none",
    ),
)

trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 2
   \\   /|    Num examples = 123,308 | Num Epochs = 1 | Total steps = 964
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 16
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 16 x 1) = 128
 "-____-"     Trainable parameters = 33,030,144 of 4,040,967,680 (0.82% trained)


Step,Training Loss


In [ ]:
model = model.merge_and_unload()
print("Адаптеры успешно слиты.")

# Сохраняем итоговую модель для инференса.
output_model_path = "qwen2-4b-recipes-finetuned-float16"
model.save_pretrained(output_model_path)
tokenizer.save_pretrained(output_model_path)

print(f"Полноценная модель в float16 сохранена в папку: {output_model_path}")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Загружаем нашу сохраненную, полноценную модель
output_model_path = "qwen2-4b-recipes-finetuned-float16" # Убедитесь, что путь верный
model_for_inference = AutoModelForCausalLM.from_pretrained(output_model_path, torch_dtype=torch.float16, device_map="auto")
tokenizer_for_inference = AutoTokenizer.from_pretrained(output_model_path)


# --- Формируем тестовый промпт в формате messages ---
test_example_input_basket = ['булочки для бургеров', 'кетчуп', 'горчица', 'лук репчатый', 'помидоры']
current_basket_str = "; ".join(test_example_input_basket)
ground_truth_for_example = "['говяжий фарш', 'сыр чеддер', 'листья салата', 'маринованные огурцы', 'картофель фри']"

# Создаем контент для юзер-промпта
user_content_for_inference = USER_PROMPT_TEMPLATE.format(current_basket=current_basket_str)

# Собираем сообщения для инференса (без ответа ассистента)
messages_for_inference = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": user_content_for_inference},
]

# Используем токенизатор для преобразования.
# add_generation_prompt=True, так как мы хотим, чтобы модель начала генерировать свой ответ.
inputs = tokenizer_for_inference.apply_chat_template(
    messages_for_inference,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt"
).to(model_for_inference.device)


# --- Генерируем ответ ---
outputs = model_for_inference.generate(input_ids = inputs, max_new_tokens = 128, use_cache = True)
# Декодируем только сгенерированную часть, исключая входной промпт
decoded_output = tokenizer_for_inference.batch_decode(outputs[:, inputs.shape[1]:], skip_special_tokens=True)[0]


# --- Печатаем результат ---
print("--- Входная корзина ---")
print(test_example_input_basket)
print("\n--- Пример правильного ответа (для сравнения) ---")
print(ground_truth_for_example)
print("\n--- Сгенерированный ответ моделью ---")
print(decoded_output.strip())